<a href="https://colab.research.google.com/github/vasusriv18-a11y/sales-analytics-system/blob/main/file_handler_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
def read_sales_data(file_path):
    """
    Reads sales data from file handling encoding issues.

    Returns:
        list: list of raw lines (strings)

    Expected Output Format:
        ['T001|2024-12-01|P101|Laptop|2|45000|C001|North', ...]

    Requirements:
    - Use 'with' statement
    - Handle different encodings (try 'utf-8', 'latin-1', 'cp1252')
    - Handle FileNotFoundError with appropriate error message
    - Skip the header row
    - Remove empty lines
    """

    encodings = ['utf-8', 'latin-1', 'cp1252']

    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                lines = file.readlines()

                # Skip header and remove empty lines
                data_lines = [
                    line.strip()
                    for line in lines[1:]
                    if line.strip() != ""
                ]

                return data_lines

        except UnicodeDecodeError:
            # Try next encoding
            continue

        except FileNotFoundError:
            print("Error: File not found. Please check the file path.")
            return []

    print("Error: Unable to read file with supported encodings.")
    return []

In [4]:
def parse_transactions(raw_lines):
    """
    Parses raw lines into clean list of dictionaries

    Returns:
        list: list of dictionaries with keys:
        ['TransactionID', 'Date', 'ProductID', 'ProductName',
         'Quantity', 'UnitPrice', 'CustomerID', 'Region']
    """

    transactions = []

    for line in raw_lines:
        # Split by pipe delimiter
        parts = line.split('|')

        # Skip rows with incorrect number of fields
        if len(parts) != 8:
            continue

        try:
            transaction_id = parts[0].strip()
            date = parts[1].strip()
            product_id = parts[2].strip()

            # Handle commas in ProductName
            product_name = parts[3].replace(',', '').strip()

            # Remove commas and convert Quantity to int
            quantity = int(parts[4].replace(',', '').strip())

            # Remove commas and convert UnitPrice to float
            unit_price = float(parts[5].replace(',', '').strip())

            customer_id = parts[6].strip()
            region = parts[7].strip()

            transaction = {
                'TransactionID': transaction_id,
                'Date': date,
                'ProductID': product_id,
                'ProductName': product_name,
                'Quantity': quantity,
                'UnitPrice': unit_price,
                'CustomerID': customer_id,
                'Region': region
            }

            transactions.append(transaction)

        except ValueError:
            # Skip rows with type conversion issues
            continue

    return transactions


In [5]:
def validate_and_filter(transactions, region=None, min_amount=None, max_amount=None):
    """
    Validates transactions and applies optional filters
    """

    required_fields = [
        'TransactionID', 'Date', 'ProductID', 'ProductName',
        'Quantity', 'UnitPrice', 'CustomerID', 'Region'
    ]

    valid_transactions = []
    invalid_count = 0

    # ---- Display available regions ----
    regions = sorted({t.get('Region') for t in transactions if 'Region' in t})
    print("Available Regions:", regions)

    # ---- Display transaction amount range ----
    amounts = [
        t['Quantity'] * t['UnitPrice']
        for t in transactions
        if 'Quantity' in t and 'UnitPrice' in t
           and isinstance(t['Quantity'], (int, float))
           and isinstance(t['UnitPrice'], (int, float))
    ]

    if amounts:
        print(f"Transaction Amount Range: {min(amounts)} to {max(amounts)}")

    total_input = len(transactions)

    # ---- Validation Step ----
    for t in transactions:
        try:
            # Check all required fields
            if not all(field in t for field in required_fields):
                invalid_count += 1
                continue

            # Business validations
            if t['Quantity'] <= 0 or t['UnitPrice'] <= 0:
                invalid_count += 1
                continue

            if not t['TransactionID'].startswith('T'):
                invalid_count += 1
                continue

            if not t['ProductID'].startswith('P'):
                invalid_count += 1
                continue

            if not t['CustomerID'].startswith('C'):
                invalid_count += 1
                continue

            valid_transactions.append(t)

        except Exception:
            invalid_count += 1

    print("Valid transactions after validation:", len(valid_transactions))

    # ---- Filtering Step ----
    filtered_by_region = 0
    filtered_by_amount = 0

    # Region filter
    if region:
        before = len(valid_transactions)
        valid_transactions = [
        t for t in valid_transactions if t['Region'] == region
        ]
        filtered_by_region = before - len(valid_transactions)
        print(f"After region filter ({region}):", len(valid_transactions))

    # Amount filter
    if min_amount is not None or max_amount is not None:
        before = len(valid_transactions)
        temp = []

        for t in valid_transactions:
            amount = t['Quantity'] * t['UnitPrice']

            if min_amount is not None and amount < min_amount:
                continue
            if max_amount is not None and amount > max_amount:
                continue

            temp.append(t)

        valid_transactions = temp
        filtered_by_amount = before - len(valid_transactions)
        print("After amount filter:", len(valid_transactions))

    # ---- Summary ----
    filter_summary = {
        'total_input': total_input,
        'invalid': invalid_count,
        'filtered_by_region': filtered_by_region,
        'filtered_by_amount': filtered_by_amount,
        'final_count': len(valid_transactions)
    }

    return valid_transactions, invalid_count, filter_summary
